In [2]:
# import necessary libraries
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import time

import opendatasets as od
import pandas as pd
import os
import shutil
from sklearn.model_selection import train_test_split

In [3]:
BATCH_SIZE = 32
IMAGE_SIZE = (128,128)
dataset_dir = './defungi'
output_dir = './Train_test_split'

#Split into train, test, validation
splitfolders.ratio(dataset_dir, output=output_dir, seed=42, ratio=(.6, .2, .2), group_prefix=None)

NameError: name 'splitfolders' is not defined